In [4]:
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.utils.capture import capture_output
from IPython import get_ipython


@magics_class
class CaptureMagic(Magics):
    @cell_magic
    def to_md(self, line, cell):
        with capture_output(stdout=True, stderr=False, display=False) as result:
            self.shell.run_cell(cell)
            message = result.stdout

        print(message)
        print("#### Markdown #### \n")
        markdown_string = ""
        markdown_string += "```python\n"
        markdown_string += cell
        markdown_string += "```\n"

        if len(message) != 0:
            markdown_string += "```console\n"
            markdown_string += "# out:\n"
            markdown_string += message
            markdown_string += "```\n"

        print(markdown_string)


ipy = get_ipython()
ipy.register_magics(CaptureMagic)


In [5]:
%%to_md
print("hi")
print(4 + 4)

hi
8

#### Markdown #### 

```python
print("hi")
print(4+4)
```
```console
# out:
hi
8
```



In [6]:
%%to_md
pass


#### Markdown #### 

```python
pass
```

